In [2]:
# !pip install transformers datasets

In [23]:
!pip install tqdm torch

In [40]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModel


model_checkpoint = "DeepPavlov/distilrubert-tiny-cased-conversational-v1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModel.from_pretrained(model_checkpoint)


/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrases/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re
from tqdm import tqdm

In [4]:
class Dataset:
    
    def __init__(self) -> None:
        self.ids = []
        self.contexts = []
        self.questions = []
        self.answers = []
        self.spans = []
ds = Dataset()

In [42]:
id = 0
contexts = raw_datasets['train']['context'] + raw_datasets['validation']['context']
questions = raw_datasets['train']['context'] + raw_datasets['validation']['question']
answers = raw_datasets['train']['context'] + raw_datasets['validation']['answers']
for i in tqdm(range(len(contexts))):

    context = contexts[i]
    answer = answers[i]
    try:
        r =  re.search(answer, context)
    
        s, e = r.span()
        ds.ids.append(id)
        id += 1
        ds.contexts.append(context)
        ds.questions.append(questions[i])
        ds.answers.append(answer)
        ds.spans.append({'start': s, 'end': e})
    except:
        None


100%|██████████| 98169/98169 [00:16<00:00, 6029.03it/s]


In [ ]:
from collections import defaultdict
import torch
hidden_dim = 264

In [78]:
input_ids = tokenizer(context,  return_tensors="pt")
last_hidden_state = model(**input_ids).last_hidden_state

In [100]:

token_id2hiddens = defaultdict(list)
for token_num, token_id in enumerate(input_ids['input_ids'][0]):
    token_id2hiddens[token_num].append(last_hidden_state[0][token_num])
token_id2hidden = {} 
for token_id, hiddens in token_id2hiddens.items():
    token_id2hidden[token_id] = torch.sum(torch.cat(hiddens).reshape((len(hiddens), hidden_dim)), axis=0)